In [1]:
import pandas as pd 

In [135]:
data=pd.read_csv("E://LearningAndExploring/Data Science/Data/Fake-New-LSTM-master/train.csv")

In [136]:
data.dropna(inplace=True)
data.reset_index(inplace=True,drop=True)

In [137]:
data.head() 

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [138]:
y=data['label']

In [139]:
data=data['title']

In [140]:
data.head() 

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                    Why the Truth Might Get You Fired
3    15 Civilians Killed In Single US Airstrike Hav...
4    Iranian woman jailed for fictional unpublished...
Name: title, dtype: object

In [141]:
data.shape 

(18285,)

In [142]:
y.shape

(18285,)

In [11]:
import nltk

In [13]:
from nltk.corpus import stopwords

In [14]:
from nltk.stem import WordNetLemmatizer

In [16]:
import re 

In [15]:
wnl=WordNetLemmatizer() 

In [143]:
corpus=[] 

In [144]:
data.shape

(18285,)

In [145]:
data.tail()

18280    Rapper T.I.: Trump a ’Poster Child For White S...
18281    N.F.L. Playoffs: Schedule, Matchups and Odds -...
18282    Macy’s Is Said to Receive Takeover Approach by...
18283    NATO, Russia To Hold Parallel Exercises In Bal...
18284                            What Keeps the F-35 Alive
Name: title, dtype: object

In [146]:
for i in range(len(data)):
    words=re.sub('[^a-zA-Z]',' ',data[i])
    words=words.lower()
    words=words.split() 
    words=[wnl.lemmatize(word) for word in words if word not in set(stopwords.words('english'))]
    words=' '.join(words)
    corpus.append(words)

In [147]:
len(corpus)

18285

In [48]:
from keras.preprocessing.text import one_hot

In [148]:
one_hotted=[one_hot(word,5000) for word in corpus]

356

In [69]:
from keras.preprocessing.sequence import pad_sequences

In [149]:
padded_sequences=pad_sequences(sequences=one_hotted,maxlen=20)

In [150]:
padded_sequences[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3966,
       3010, 3446, 2013, 1881,  314, 2865,   15, 1607, 1186])

In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import Dropout

Using TensorFlow backend.


In [2]:
from keras.layers import LSTM

In [3]:
classifier=Sequential() 

In [4]:
classifier.add(Embedding(5000,40,input_length=20))

In [175]:
classifier.add(Dropout(0.2))


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [176]:
classifier.add(LSTM(100))

In [177]:
classifier.add(Dropout(0.2))

In [178]:
classifier.add(Dense(1,activation='sigmoid'))

In [179]:
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [180]:
classifier.summary() 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 40)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [103]:
import numpy as np 

In [151]:
X_final = np.array(padded_sequences)

In [152]:
y_final = np.array(y)

In [153]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [155]:
from sklearn.model_selection import train_test_split

In [156]:
 X_train, X_test, y_train, y_test = train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [185]:
classifier.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=7,batch_size=64)

Train on 12250 samples, validate on 6035 samples
Epoch 1/7
12250/12250 [==============================] - 13s 1ms/step - loss: 0.0086 - acc: 0.9976 - val_loss: 0.5851 - val_acc: 0.9089
Epoch 2/7
12250/12250 [==============================] - 12s 1ms/step - loss: 0.0079 - acc: 0.9980 - val_loss: 0.5482 - val_acc: 0.9082
Epoch 3/7
12250/12250 [==============================] - 12s 992us/step - loss: 0.0059 - acc: 0.9987 - val_loss: 0.5176 - val_acc: 0.9051
Epoch 4/7
12250/12250 [==============================] - 12s 1ms/step - loss: 0.0047 - acc: 0.9988 - val_loss: 0.6564 - val_acc: 0.9060
Epoch 5/7
12250/12250 [==============================] - 14s 1ms/step - loss: 0.0044 - acc: 0.9990 - val_loss: 0.6735 - val_acc: 0.9087
Epoch 6/7
12250/12250 [==============================] - 13s 1ms/step - loss: 0.0056 - acc: 0.9984 - val_loss: 0.5124 - val_acc: 0.9075
Epoch 7/7
12250/12250 [==============================] - 13s 1ms/step - loss: 0.0094 - acc: 0.9969 - val_loss: 0.5250 - val_acc: 0.90

In [182]:
y_pred=classifier.predict_classes(X_test)

In [158]:
from sklearn.metrics import confusion_matrix

In [183]:
confusion_matrix(y_test,y_pred)

array([[3122,  297],
       [ 261, 2355]], dtype=int64)

In [161]:
from sklearn.metrics import accuracy_score

In [184]:
accuracy_score(y_pred,y_test)

0.9075393537696769